<a href="https://colab.research.google.com/github/DreRnc/ExplainingExplanations/blob/main/Explanations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset : **E-SNLI**. \
Model : **Small T5**.

In [ ]:
colab = True

In [ ]:
if colab:
    !git clone https://github.com/DreRnc/ExplainingExplanations.git
    %cd ExplainingExplanations
    %pip install -r requirements.txt

# 1.0 Preparation


## 1.1 Loading Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("esnli")

In [ ]:
training_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

print("Shape of training_set: ", training_set.shape)
print("Shae of validation_set: ", validation_set.shape)
print("Shape of test_set: ", test_set.shape)

In [ ]:
training_set[0]

In [ ]:
n_train = n_valid = n_test = 500

train_small = training_set.select(range(n_train))
valid_small = validation_set.select(range(n_valid))
test_small = test_set.select(range(n_test))

print("Shape of train_small: ", train_small.shape)
print("Shape of valid_small: ", valid_small.shape)
print("Shape of test_small: ", test_small.shape)

## 1.2 Loading T5 Model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

Test **zero-shot** on a random task.

In [ ]:
input_ids = tokenizer(
    "translate English to French: Hello Dre, I think the English version is ok for us.",
    return_tensors="pt",
).input_ids
outputs = model.generate(input_ids, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=100))

## 1.3 Zero-shot example to Verify Everything is Working

In [ ]:
from src.utils import generate_prompt_mnli

In [ ]:
example = training_set[0]
example

Generating the prompt:

<b><u> mnli hypothesis: </b></u> The St. Louis Cardinals have always won. <b><u> premise: </b></u> yeah well losing is i mean i’m i’m originally from Saint Louis and Saint Louis Cardinals when they were there were uh a mostly a losing team but

Output:
* 0: Entailment
* 1: Neutral
* 2: Contradiction

In [ ]:
prompt = generate_prompt_mnli(example)
prompt

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 1.4  Tokenize the dataset

In [ ]:
train_small.info

In [ ]:
train_small.features

In [ ]:
from functools import partial
from src.utils import tokenize_function
import time

In [ ]:
tokenize_mapping = partial(tokenize_function, tokenizer=tokenizer)

In [ ]:
time_init = time.time()
train_small_tokenized = train_small.map(tokenize_mapping, batched=True).with_format("torch")
valid_small_tokenized = valid_small.map(tokenize_mapping, batched=True).with_format("torch")
test_small_tokenized = test_small.map(tokenize_mapping, batched=True).with_format("torch")
time_end = time.time()

print("Time taken to tokenize: ", time_end - time_init)
print("Shape of train_small_tokenized: ", train_small_tokenized.shape)
print("Shape of valid_small_tokenized: ", valid_small_tokenized.shape)
print("Shape of test_small_tokenized: ", test_small_tokenized.shape)

In [ ]:
train_small_tokenized.features

# 2.0 Task 1: Zero-shot evaluation

In [ ]:
from src.utils import evaluate_output_mnli
from tqdm import tqdm
import torch

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model.to(device)
device

In [21]:
correct = 0
seen = 0
batch_size = 256 # Choose an appropriate batch size

for i in tqdm(range(0, len(train_small_tokenized), batch_size)):
    batch_dict = train_small_tokenized[i : i + batch_size]

    # Prepare input_ids tensor for the entire batch
    input_ids = batch_dict["input_ids"].to(device)

    # Generate outputs for the batch
    outputs = model.generate(input_ids)

    # Iterate through the batch and evaluate each output
    for idx, output_ids in enumerate(outputs):
        output = tokenizer.decode(output_ids, skip_special_tokens=True)
        correct += evaluate_output_mnli(output, batch_dict["label"][idx])
        seen += 1

print("Accuracy zero-shot on the test set: ", correct / seen)

100%|██████████| 20/20 [00:22<00:00,  1.10s/it]

Accuracy zero-shot on the test set:  tensor(0.7068)


# 3.0 Task 2: Fine tuning without explanations

In [ ]:
import numpy as np
import evaluate
from src.utils import compute_accuracy
from transformers import TrainingArguments, Trainer, AutoTokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model_ft = T5ForConditionalGeneration.from_pretrained("t5-small")

In [30]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=train_small_tokenized,
    eval_dataset=test_small_tokenized,
    compute_metrics=compute_accuracy
)

In [ ]:
#trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

# 4.0 Task 4: Making the model generate explanations